In [1]:
print("Hello!")

Hello!


In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [2]:
import base64


def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

In [3]:
from datasets import load_dataset

ds = load_dataset("Rapidata/text-2-image-Rich-Human-Feedback", split="train", streaming=True)


Resolving data files:   0%|          | 0/54 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/54 [00:00<?, ?it/s]

In [4]:
from io import BytesIO

from PIL import Image
import base64
def pil_to_base64(image: Image.Image) -> str:
    buffer = BytesIO()
    image.save('output.jpg', format="JPEG")
    image.save(buffer, format="JPEG")  # Ensure the format is set to JPEG
    base64_str = base64.b64encode(buffer.getvalue()).decode("utf-8")
    return base64_str

In [ ]:
ds_words = ds.select_columns(["image", "prompt", "alignment_score", "coherence_score", "word_scores"])
for example in ds_words.take(500):
    if example["alignment_score"]  < 2.5:
        result = example
        print(example["alignment_score"], example["prompt"])
        pil_to_base64(example["image"])
        break

In [17]:
print(result["word_scores"])

[["A", 0.9753], ["sign", 2.6966], ["that", 0.9753], ["says", 1.8856], ["'Diffusion'.", 8.1711], ["[No_mistakes]", 10.2536]]


In [15]:
from itertools import islice

# Assuming you have an IterableDataset
dataset = ds_words

# Get the 3rd element (0-based index, so index 2)
third_element = next(islice(dataset, 63, 64))

In [16]:
third_element['alignment_score']

3.1654999256134033

In [17]:
third_element['prompt']

'a landscaping company called norwegian fjord'

In [18]:
base64_image = pil_to_base64(third_element["image"])


In [ ]:
scores = []
from tqdm import tqdm
for example in tqdm(ds_words):
    scores.append(example['alignment_score'])

import numpy as np
print(np.min(scores))

41it [00:28,  1.76it/s]Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7fe751dbabc0>>
Traceback (most recent call last):
  File "/home/luckyman/.local/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 
71it [00:41,  2.09it/s]

In [5]:
# Load and preprocess image
import torch
import open_clip
from PIL import Image

# Load CLIP model

def clip_score(example):
    model, _, preprocess = open_clip.create_model_and_transforms('ViT-B-32', pretrained='openai')
    tokenizer = open_clip.get_tokenizer('ViT-B-32')
    image = preprocess(example["image"]).unsqueeze(0)

    # Tokenize text
    text = tokenizer(example["prompt"])

    # Move to available device (CPU/GPU)
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model = model.to(device)
    image = image.to(device)
    text = text.to(device)

    # Compute embeddings
    with torch.no_grad():
        image_features = model.encode_image(image)
        text_features = model.encode_text(text)

    # Normalize features
    image_features /= image_features.norm(dim=-1, keepdim=True)
    text_features /= text_features.norm(dim=-1, keepdim=True)

    # Compute cosine similarity
    similarity = 4*((image_features @ text_features.T).item())+1

    print(f"Similarity Score: {similarity:.4f}")
    return similarity


In [9]:
clip_score(result)

/home/luckyman/.local/lib/python3.10/site-packages/open_clip/factory.py:388: UserWarning: These pretrained weights were trained with QuickGELU activation but the model config does not have that enabled. Consider using a model config with a "-quickgelu" suffix or enable with a flag.
  warnings.warn(


Similarity Score: 2.0833


2.0832602977752686

In [18]:
print(llm_score(element=result))

5


In [49]:
clip_metric = []
human_metric = []

for element in islice(dataset, 0, 10):
    base64_image = pil_to_base64(element["image"])
    clip_metric.append(clip_score(element["prompt"]))
    human_metric.append(element["alignment_score"])


/home/luckyman/.local/lib/python3.10/site-packages/open_clip/factory.py:388: UserWarning: These pretrained weights were trained with QuickGELU activation but the model config does not have that enabled. Consider using a model config with a "-quickgelu" suffix or enable with a flag.
  warnings.warn(


Similarity Score: 2.0694
Similarity Score: 2.3566
Similarity Score: 2.3202
Similarity Score: 2.0886
Similarity Score: 2.3886
Similarity Score: 2.2289
Similarity Score: 2.5424
Similarity Score: 2.3901
Similarity Score: 2.1493
Similarity Score: 2.2739


In [21]:
elements = next(islice(dataset, 0, 10))
print(elements)

{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1024x1024 at 0x7FCD6D3BF970>, 'prompt': 'The bright green grass contrasted with the dull grey pavement.', 'alignment_score': 3.45740008354187, 'coherence_score': 3.596299886703491}


In [6]:
human_prompt_template = """
Task description:
In this task you will evaluate 3 metrics of image generated by AI.
Metrics: Plausability of image, Text-Alignment(overall alignment image to promp), Aesthetics 
To correctly make evaluation follow the INSTRUCTIONS
**INSTRUCTIONS**:
1. Understand the Textual Prompt
    a. Extract the core meaning of the prompt.
    b. Identify key elements such as objects, actions, relationships, emotions, colors, styles, or any specific details mentioned.
2. Analyze the Image
    a. Identify the main subjects, objects, actions, and visual characteristics of the image.
    b. Consider elements such as composition, colors, textures, and details that align with the prompt.
3. Rate image in 3 metrics based on evaluation criteria(0 worst, 1 best)

**Evaluation critera**
#### Evaluation criteria for Text Alignemt
1. Check if the main objects and actions in the prompt appear in the image.
2. Assess semantic alignment (e.g., if the prompt describes a "red apple," ensure a red apple is present rather than a green one).
3. Consider stylistic and contextual alignment (e.g., if the prompt asks for "a realistic portrait," but the image is abstract, the alignment is lower).

1.0 → Perfect match (The image fully matches the prompt in all aspects).
0.8 - 0.9 → High alignment (Minor deviations in details but the overall meaning is correct).
0.5 - 0.7 → Partial match (Some elements are missing or incorrect, but the general idea is preserved).
0.2 - 0.4 → Low alignment (The image and prompt share few relevant elements).
0.0 - 0.1 → No alignment (The image does not represent the prompt at all).

#### Evaluation criteria for Plausability
1. Check if image looks natural and free from visible artifacts
2. Check if elements on image are logically aligned
3. Are textures, facial features, and small elements consistent and believable?

1.0 → Perfect match (The image looks really natural without any visible artifacts).
0.8 - 0.9 → High alignment (Image has small artifact or some objects are misaligned).
0.5 - 0.7 → Partial match (The image has noticeable artifacts or unnatural elements.).
0.2 - 0.4 → Low alignment (The image has multiple artifacts or unnatural distortions.).
0.0 - 0.1 → No alignment (The image is heavily distorted or unnatural.).

#### Evaluation criteria for Aesthetics
1. Check overall image sharpness and quality
2. Analyze color palette cohesiveness and appropriateness
3. Check how well looking is image at all

"""

In [17]:
print(third_element['alignment_score'], third_element['coherence_score'])

4.046199798583984 4.280099868774414


In [63]:
llm_metric = []
human_metric = []

for idx, element in enumerate(islice(dataset, 0, 10)):
    base64_image = pil_to_base64(element["image"])
    print(idx, element['prompt'])
    score = float(llm_score(element=element))
    
    score = 4*score+1
    print(score)
    llm_metric.append(float(score))
    human_metric.append(element["alignment_score"])

import scipy.stats as stats
kendall_tau, _ = stats.kendalltau(llm_metric, human_metric)
print(kendall_tau)

0 The bright green grass contrasted with the dull grey pavement.
4.6
1 image from an iPhone video of a dog in a supermarket, hyper realistic, flash photo
5.0
2 A man wearing a brown cap looking sitting at his computer with a black and brown dog resting next to him on the couch.
5.0
3 A beige pastry sitting in a white ball next to a spoon .
4.2
4 a diverse crowd of people eagerly waits in line at a bustling street food stand in beirut. the tantalizing aroma of freshly grilled kebabs and warm pita bread fills the air. the stand is adorned with vibrant lanterns and colorful signs showcasing the mouthwatering menu. nearby, a street musician plays an upbeat tune on his oud, adding to the lively atmosphere.
4.6
5 photograph of a person drinking red wine and smoking weed with a flat cigarette
5.0
6 A yellow horse and a red chair.
5.0
7 A guitar made of ice cream that melts as you play it.
5.0
8 a fluffy pillow and a leather belt
5.0
9 hyperrealism fruits and vegetables market
5.0
0.3419184296

In [64]:
human_metric

[3.45740008354187,
 3.4003000259399414,
 4.132400035858154,
 2.8675999641418457,
 3.7504000663757324,
 3.6187000274658203,
 3.5425000190734863,
 3.8287999629974365,
 3.3171000480651855,
 3.9697999954223633]

In [65]:
llm_metric

[4.6, 5.0, 5.0, 4.2, 4.6, 5.0, 5.0, 5.0, 5.0, 5.0]

In [50]:
import scipy.stats as stats
kendall_tau, _ = stats.kendalltau(clip_metric, human_metric)
print(kendall_tau)

0.3333333333333333


In [ ]:
prev_version_prompt = """ You are helpful assistant. You will be provided with description and image. Your goal is to determine how good image align with the description. 
                            Follow instructions step by step and verify each step.
                            Instruction:
                            Steps to Evaluate Alignment

                            1. Understand the Textual Prompt
                                a. Extract the core meaning of the prompt.
                                b. Identify key elements such as objects, actions, relationships, emotions, colors, styles, or any specific details mentioned.

                            2. Analyze the Image
                                a. Identify the main subjects, objects, actions, and visual characteristics of the image.
                                b. Consider elements such as composition, colors, textures, and details that align with the prompt.

                            3. Compare Textual and Visual Features
                                a. Check if the main objects and actions in the prompt appear in the image.
                                b. Assess semantic alignment (e.g., if the prompt describes a "red apple," ensure a red apple is present rather than a green one).
                                c. Consider stylistic and contextual alignment (e.g., if the prompt asks for "a realistic portrait," but the image is abstract, the alignment is lower).

                            4. Assign an Alignment Score (0-1 Scale)
                                1.0 → Perfect match (The image fully matches the prompt in all aspects).
                                0.8 - 0.9 → High alignment (Minor deviations in details but the overall meaning is correct).
                                0.5 - 0.7 → Partial match (Some elements are missing or incorrect, but the general idea is preserved).
                                0.2 - 0.4 → Low alignment (The image and prompt share few relevant elements).
                                0.0 - 0.1 → No alignment (The image does not represent the prompt at all).
                            5. As response return ONLY Alignment score as number without any additional thoughts and comments
                            """